# RRAP-IS Provenance Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

## About

This notebook is a demonstration of generating a record of a model run and searching the provenance database to identify linkages between data, models, users and organisations with the RRAP-IS system.  

### Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [2]:
%%capture
import requests
import os
import sys
import json
from json2html import *
from bs4 import BeautifulSoup
from IPython.display import IFrame, display, HTML, JSON, Markdown, Image
from mdsisclienttools.auth.TokenManager import DeviceFlowManager
from urllib.error import HTTPError
import networkx as nx
import nx_altair as nxa
from networkx.readwrite import json_graph

import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
defaults = dict(width=800, height=600)
hv.opts.defaults(opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

import warnings
warnings.filterwarnings(action='once')

### Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [3]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit()
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit()

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


## Authentication

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

[Return to Top](#toc)

In [4]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Using storage type: FILE.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Trying to use found tokens to refresh the access token.

Tokens found in storage but they are not valid.

Initiating device auth flow to setup offline access token.

Decoding response

Please authorise using the following endpoint.

Verification URL: https://auth.dev.rrap-is.com/auth/realms/rrap/device?user_code=KIRM-TXCI
User Code: KIRM-TXCI

Awaiting completion


Token generation complete. Authorisation successful.



/home/ross/repos/rrap-demo-blog/.venv/lib/python3.10/site-packages/mdsisclienttools/auth/TokenManager.py:407: ResourceWarning: unclosed <ssl.SSLSocket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.31.4.212', 44126), raddr=('54.253.194.144', 443)>
  oauth_tokens = self.await_device_auth_flow_completion(


## Helper functions

[Return to Top](#toc)

In [5]:
def wrap_html_table(data):
    soup = BeautifulSoup(data)

    ul_tag = soup.find("table")
    div_tag = soup.new_tag("div")
    div_tag['style'] = "width: auto; height: 400px; overflow-y: auto; "
    ul_tag.wrap(div_tag)
    new_tag = soup.new_tag("details")
    div_tag.wrap(new_tag)
    
    tag = soup.new_tag("summary")
    tag.string = "Results"
    soup.div.insert_after(tag)

    return soup.prettify()
    
def json_to_md(response_json):
        json_obj_in_html = json2html.convert( response_json  )
        return wrap_html_table(json_obj_in_html)
    
def handle_request(method, url, params=None, payload=None, auth=None):
    try:
        if params:
            response = requests.request(method, url=url, params=params, auth=auth)
        elif payload:
            response = requests.request(method, url=url, json=payload, auth=auth)
        else:
            response = requests.request(method, url=url, auth=auth)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
        return response.json()
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
        return response.json()
    else:
        return response.json()

## Endpoint Documentation
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
<ul>
  <li><a href="https://prov-api.testing.rrap-is.com/redoc" target="_blank">Redoc FastAPI</a></li>
  <li><a href="https://prov-api.testing.rrap-is.com/docs" target="_blank">Docs FastAPI</a> </li>
</ul>

[Return to Top](#toc)

## Demonstration

### Register a model run
A model run consists of ten main elements where many have already been registered in the RRAP-IS Registry.  

These elements are:
- Start Time
- End Time
- Workflow definition*
- Input dataset*
- Input template*
- Output dataset*
- Output template*
- Modeller* and
- Requesting Organisation*

*pre-registered elements.

For examples of registering elements please see <a href="/rrap-demo-blog/jupyter/2022/08/18/rrap-registry.html" target="_blank">RRAP-IS Register blog post</a> or <a href="/rrap-demo-blog/jupyter/registry/2022/09/02/registry-query.html" target="_blank">RRAP-IS Register Query blog post</a>

see <a href="https://prov-api.testing.rrap-is.com/redoc#tag/Model-Runs/operation/register_model_run_complete_model_run_register_complete_post" target="_blank">Endpoint documentation</a>

In [37]:
from enum import Enum
from enum_switch import Switch
class ProvType(Enum):
    AGENT = 1
    ACTIVITY = 2
    ENTITY = 3

class ItemType(Enum):
    MODEL = 1
    PERSON = 2
    ORGANISATION = 3
    MODELRUN = 4
    MODEL_RUN_WORKFLOW = 5
    DATASET_TEMPLATE = 6

class ProvTypeFromItemType(Switch):
    def MODEL(self):
        return ProvType.ENTITY

    def PERSON(self):
        return ProvType.AGENT    

    def ORGANISATION(self):
        return ProvType.AGENT    

    def MODELRUN(self):
        return ProvType.ACTIVITY

    def MODEL_RUN_WORKFLOW(self):
        return ProvType.ENTITY
    
    def DATASET_TEMPLATE(self):
        return ProvType.ENTITY

prov_of_item = ProvTypeFromItemType(ItemType)

In [38]:
def register_item(payload, item_type, auth):
    prov_type = prov_of_item(item_type)
    postfix = f'/registry/{prov_type.name.lower()}/{item_type.name.lower()}/create'
    endpoint = registry_api + postfix 
    return requests.post(endpoint, json=payload, auth=auth())

In [40]:
person_payload = {
    "display_name": "Ross Petridis",
    "first_name": "Ross",
    "last_name": "Petridis",
    "email": "ross.petridis@csiro.au ",
    "orcid": "https://orcid.org/0000-0002-3798-9719"
    }
auth = token_manager.get_auth
response = register_item(person_payload, ItemType.PERSON, auth)
print(response.json())
person_id = response.json()["created_item"]["id"]

Token validation failed due to error: Signature has expired.
Refreshing using refresh token

{'status': {'success': True, 'details': 'Successfully uploaded the complete item. Return item includes handle id.'}, 'created_item': {'display_name': 'Ross Petridis', 'email': 'ross.petridis@csiro.au', 'first_name': 'Ross', 'last_name': 'Petridis', 'orcid': 'https://orcid.org/0000-0002-3798-9719', 'id': '10378.1/1695722', 'created_timestamp': 1669855699, 'updated_timestamp': 1669855699, 'item_category': 'AGENT', 'item_subtype': 'PERSON', 'record_type': 'COMPLETE_ITEM'}}


KeyError: 'id'

In [46]:
model_payload = {
    "display_name": "ADRIA",
    "name": "ADRIA",
    "description": "Coral Community Network",
    "documentation_url": "https://github.com/gbrrestoration/CoCoNet-model/blob/main/README.md",
    "source_url": "https://gbrrestoration.org/"
    }
auth = token_manager.get_auth
response = register_item(model_payload, ItemType.MODEL, auth)
model_id = response.json()["created_item"]['id']

In [47]:
template_payload = {
  "display_name": "Example dataset template",
  "description": "Description for example dataset template",
  "defined_resources": [],
  "deferred_resources": []
}


response = register_item(template_payload, ItemType.DATASET_TEMPLATE, auth)
dataset_template_id = response.json()['created_item']['id']


In [48]:

modelrun_workflow_template_paylaod = {
  "display_name": "Workflow def using ADRIA model (software)",
  "software_id": model_id,
  "software_version": "1.0",
  "annotations": {
    "required": [],
    "optional": [],
  },
  "input_templates": [
    {
      "template_id" : dataset_template_id
    }
  ],
  "output_templates": [
    {
      "template_id" : dataset_template_id
    }
  ]
}

auth = token_manager.get_auth
response = register_item(modelrun_workflow_template_paylaod, ItemType.MODEL_RUN_WORKFLOW, auth)
workflow_def_id = response.json()['created_item']['id']

In [57]:
auth = token_manager.get_auth
postfix = "/model_run/register_complete"
payload = {
  "start_time": 0,
  "end_time": 1662467929,
  "workflow_template_id": workflow_def_id,

  "inputs": [
    {
      "dataset_template_id" : dataset_template_id,
      "dataset_id" : "10378.1/1691958",
      "dataset_type" : "DATA_STORE",
      "resources" : {
        
      } 
    }
  ],

  "outputs": [
    {
      "dataset_template_id" : dataset_template_id,
      "dataset_id" : "10378.1/1691958",
      "dataset_type" : "DATA_STORE",
      "resources" : {
        
      } 
    }
  ],

  "associations": {
    "modeller_id": person_id,
  }
}
endpoint = prov_api + postfix 

response_json = handle_request('POST', endpoint, None, payload, auth())
HTML(json_to_md(response_json))
model_run_id = response_json['record_info']['id']

## Exploring linkages across RRAP-IS data, models, modellers and organisation
Simple exploration functions include:
- Searching upstream (option to only looks so far) from a point and
- Searching downstream (option to only looks so far) from a point


### Explore Upstream Provenance
Lets explore back upstream from the generated output from the above completed model run.  We will limit it to a depth of one, so only the directly linked elements

[Return to Top](#toc)

In [58]:
auth = token_manager.get_auth
postfix = "/explore/upstream"
params = {
    "starting_id": model_run_id,
    "depth": 1
}

endpoint = prov_api + postfix

response_json = handle_request('GET', endpoint, params, None, auth())
result_graph = response_json["graph"]

networkx_graph = json_graph.node_link_graph(result_graph)
im = hv.Graph.from_networkx(networkx_graph, nx.layout.circular_layout).opts(tools=['hover','tap'],
                                                                          node_size=10,
                                                                          node_color='item_category',
                                                                          cmap = ['blue','orange', 'green', 'red'],
                                                                          directed=True, 
                                                                          arrowhead_length=0.02,
                                                                          bgcolor='pink')


In [59]:
im.nodes.data


,x,y,index,item_category,item_subtype
0,0.309017,0.951057,10378.1/1691958,ENTITY,DATASET
1,0.309017,-0.951057,10378.1/1695722,AGENT,PERSON
2,-0.809017,0.587785,10378.1/1695723,ENTITY,MODEL
3,-0.809017,-0.587785,10378.1/1695725,ENTITY,MODEL_RUN_WORKFLOW_TEMPLATE
4,1.000000,0.000000,10378.1/1695727,ACTIVITY,MODEL_RUN


In [62]:
labels = hv.Labels(im.nodes, ['x', 'y'], 'item_category').opts(opts.Labels(text_font_size='12pt', text_color='blue', xoffset=0, yoffset=0.05, bgcolor='white'))


In [63]:
labels_2 = hv.Labels(im.nodes, ['x', 'y'], 'item_subtype').opts(opts.Labels(text_font_size='8pt', xoffset=0, yoffset=-0.05, bgcolor='white'))
hv_graph = (im * labels * labels_2)

hv.save(hv_graph, 'lineage_network.html', backend='bokeh')

HTML('lineage_network.html')

### Explore Downstream Provence
Lets explore forward downstream from the generated model definition defined above.  We will limit it to a depth of one, so only the directly linked elements

[Return to Top](#toc)

In [64]:
auth = token_manager.get_auth
postfix = "/explore/downstream"
params = {
    #"starting_id": "10378.1/1691197",
    "starting_id": model_run_id,
    "depth": 1
}
endpoint = prov_api + postfix 

response_json = handle_request('GET', endpoint, params, None, auth())


Token validation failed due to error: Signature has expired.
Refreshing using refresh token



In [24]:
print(response_json)

{'status': {'success': True, 'details': 'Made downstream query (with depth 1) to neo4j backend.'}, 'record_count': 0, 'graph': {'directed': True, 'multigraph': False, 'graph': {}, 'nodes': [], 'links': []}}


In [65]:
result_graph = response_json["graph"]

networkx_graph = json_graph.node_link_graph(result_graph)
im = hv.Graph.from_networkx(networkx_graph, nx.layout.fruchterman_reingold_layout).opts(tools=['hover','tap'],
                                                                          node_size=10,
                                                                          node_color='item_category',
                                                                          cmap = ['blue','orange', 'green', 'red'],
                                                                          directed=True, 
                                                                          arrowhead_length=0.02,
                                                                          bgcolor='pink')
labels = hv.Labels(im.nodes, ['x', 'y'], 'index').opts(opts.Labels(text_font_size='12pt', text_color='blue', xoffset=0, yoffset=0.05, bgcolor='white'))
labels_2 = hv.Labels(im.nodes, ['x', 'y'], 'index').opts(opts.Labels(text_font_size='8pt', xoffset=0, yoffset=-0.05, bgcolor='white'))
hv_graph = (im * labels * labels_2)

hv.save(hv_graph, 'network.html', backend='bokeh')

HTML("network.html")

[Return to Top](#toc)